## Semi-Supervised Learning with Spectral Graph Convolutions

### Load Zachary's Karate Club data

In [143]:
import networkx as nx
from collections import namedtuple
import pandas as pd
import numpy as np
import tensorflow as tf

In [136]:
def load_data():
    '''
    Load data into a networkx graph
    '''

    network = nx.read_edgelist(
        'data/zkc.edgelist',
        nodetype=int
        )
    
    attributes = pd.read_csv(
        'data/features.csv',
        index_col=['node']
        )

    
    for attribute in attributes.columns.values:
        nx.set_node_attributes(
                network,
                values=pd.Series(
                    attributes[attribute],
                    index=attributes.index).to_dict(),
                name=attribute
            )

    # Create a mask of all the train samples
    # We will only use the Administrator and the Instructor but none of the members
    train_mask = [True if attribute in ['Administrator', 'Instructor'] else False for attribute in attributes['role']]
    test_mask = [True if item is False else False for item in train_mask]   

    return train_mask, test_mask, network

In [137]:
train_mask, test_mask, network = load_data()

In [147]:
# Encode Label categories
attributes['community'] = attributes['community'].astype('category')
attributes['community'] = attributes['community'].cat.codes

In [164]:
# Get the adjacency matrix
adj = nx.to_numpy_array(zkc.network)
adj += np.eye(adj.shape[0])
# First, we will only use the identity matrix as the only feature
features = np.eye(adj.shape[0])

# Labels
labels = attributes['community'].values
labels = labels.reshape(34, -1)

In [165]:
features = features.astype('float32')
adj = adj.astype('float32')
labels = labels.astype('float32')

In [166]:
def masked_softmax_cross_entropy(logits, labels, mask):
    '''Applies loss function taking into account the mask to
       only take into account relevant nodes.
       Returns cross entropy loss over the masked nodes of the graph.'''
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    mask = tf.cast(mask, dtype=tf.float32)
    # Divide mask by its average value - that will enable us to take the product of the mask with the loss
    mask /= tf.reduce_mean(mask)  # Is this step used to normalise?
    loss *= mask
    # return average across all positions
    return tf.reduce_mean(loss)


def masked_accuracy(logits, labels, mask):
    '''Returns accuracy over the masked nodes of the graph.'''
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [167]:
def gnn(fts, adj, transform, activation):
    '''
    Define a Graph Neural Network layer.
    fts: node feature matrix
    adj: adjacency matrix
    transform: some transformation that we wish to apply to each node
    activation: activation function
    '''
    # Transform each node 
    seq_fts = transform(fts)
    # Once we have the features we want to aggregate we multiply by the adjacency matrix
    ret_fts = tf.matmul(adj, seq_fts)
    return activation(ret_fts)

In [176]:
def train_model(fts, adj, gnn_fn, units, epochs, lr):
    '''Define simple 2 layer GNN.
       gnn_fn: gnn model function
       units: how many units we want our NN to compute in each node - 
              how many dimentions in our latent features.'''
    lyr_1 = tf.keras.layers.Dense(units)
    # Computes classification of nodes
    lyr_2 = tf.keras.layers.Dense(1)
    
    def cora_gnn(fts, adj):
        ''' Define the GNN that is used to solve this problem
            on a specific set of features and adjacencies.'''
        # Computes hidden features in every node
        hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
        logits = gnn_fn(hidden, adj, lyr_2, tf.identity)  # We don't need any further transformation so we use the identity matrix as activation
        return logits
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    best_accuracy = 0.0
    for ep in range(epochs+1):
        # Use tape to keep track of gradients 
        with tf.GradientTape() as t:
            logits = cora_gnn(fts, adj)
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
            
        # Look at variables that gradient tape is watching
        variables = t.watched_variables()  # Get variables
        grads = t.gradient(loss, variables)  # Get gradients

        optimizer.apply_gradients(zip(grads, variables))  # Apply gradients
        
        # Track val and test accuracy
        logits = cora_gnn(fts, adj)  # Take logits after gradients have been updated
        # val_accuracy = masked_accuracy(logits, labels, val_mask)
        test_accuracy = masked_accuracy(logits, labels, test_mask)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            print('Epoch', ep, '| Training Loss:', loss.numpy(),
                  '|Test accuracy:', test_accuracy.numpy())

In [177]:
hidden_features = 32
epochs = 200
learning_rate = 0.01

train_model(features, adj, gnn, hidden_features, epochs, learning_rate)

Epoch 0 | Training Loss: 0.0 |Test accuracy: 1.0
